In [3]:
from xgboost import XGBClassifier as XGBC
import lightgbm as lgbm
import seaborn as sb
import pandas as pd 
from pandas.plotting import lag_plot, autocorrelation_plot as acplot
import matplotlib.pyplot as plt
import missingno as mano
import numpy as np
from numpy import mean
import math
import time
from math import sqrt
import warnings
from scipy.stats import skew, kurtosis, spearmanr as spm, pearsonr as pe, chi2_contingency
from scipy.special import boxcox, inv_boxcox
from sklearn.impute import KNNImputer, SimpleImputer
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.ar_model import AR
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA, ARMA
from random import gauss, seed
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
import pylab as py
from sklearn.feature_selection import RFE, RFECV, mutual_info_regression
from sklearn import svm, tree
from sklearn.svm import SVR
from sklearn.preprocessing import OrdinalEncoder as ordenc, OneHotEncoder as ohot, StandardScaler as StdSclr, MinMaxScaler as MMS
from sklearn.metrics import confusion_matrix as confm, accuracy_score as acs, classification_report as crep, log_loss as logloss, mean_absolute_error as MAE, mean_squared_error as MSE, r2_score as r2, precision_score as pscore, recall_score as rscore, roc_auc_score as rascore, f1_score as f1score, roc_curve as roccurve, auc 
from sklearn.model_selection import train_test_split as tts, KFold, RepeatedKFold as RKFold, StratifiedKFold as SKFold, StratifiedShuffleSplit as SSS, cross_val_score as cvl, LeaveOneOut as lvo, RepeatedStratifiedKFold as rskfold
from sklearn.tree import DecisionTreeClassifier as dtc, DecisionTreeRegressor as dtr
from sklearn.linear_model import LinearRegression as LR, LogisticRegression as LogR, Ridge as rdg, Lasso as lso
from sklearn.ensemble import RandomForestClassifier as RFC, GradientBoostingClassifier as GBC, AdaBoostClassifier as ABC, GradientBoostingRegressor as GBR, AdaBoostRegressor as ABR, VotingRegressor as VR, RandomForestRegressor as RFR
from sklearn.naive_bayes import GaussianNB as GNB
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier as mlpc
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures as PF
get_ipython().run_line_magic('matplotlib', 'inline')
plt.rcParams['figure.figsize'] = (20.0, 10.0)
from statistics import mean, median, mode
warnings.filterwarnings("ignore")
import graphviz
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler 
from collections import Counter
from IPython.display import Image
from __future__ import print_function

In [39]:
def deepcopy(dataset):
    return dataset.copy()

In [40]:
def cat_analysis(dataset):
    colname = str(input("which column do you want to analyze? Press enter if you want to analyze the entire dataset."))
    if colname != '':
        dataset[colname].plot()
        plt.show()
        print(dataset[colname].describe())
        fit = str(input("what type of fit do you want the qqplot to be? (None, 45, s, r, or q)\n"))
        sm.qqplot(dataset[colname], line = fit)
        plt.show()
        plt.hist(dataset[colname])
        plt.show()
        dataset[colname].plot.box()
        plt.show()
        skewvalue = skew(dataset[colname])
        print('The skewvalue is', skewvalue)
        if skewvalue>-2 and skewvalue<2:
            print("Normally skewed data")
        else:
            print("Not normally skewed data")
        kurtosis_value = kurtosis(dataset[colname])
        print(kurtosis_value)
        if kurtosis_value>-7 and kurtosis_value<7:
            print("Normal Kurtosis")
        else:
            print("Not normal Kurtosis")
    else:
        dataset.plot()
        plt.show()
        dataset.describe()
        fit = str(input("what type of fit do you want the qqplot to be? (None, 45, s, r, or q)\n"))
        sm.qqplot(dataset, line = fit)
        plt.show()
        plt.hist(dataset)
        plt.show()
        dataset.plot.box()
        plt.show()
        skewvalue = skew(dataset)
        print(skewvalue)
        if (skewvalue>-2).all() and (skewvalue<2).all():
            print("Normally skewed data")
        else:
            print("Not normally skewed data")
        kurtosis_value = kurtosis(dataset)
        print(kurtosis_value)
        if (kurtosis_value>-7).all() and (kurtosis_value<7).all():
            print("Normal Kurtosis")
        else:
            print("Not normal Kurtosis")

In [41]:
def missval(dataset):
    x = str(input("What analysis do you want to do? (missingval, totalmissingval, or correlation, dendogram)"))
    if x == 'missingval':
        return mano.matrix(dataset)
    elif x == 'totalmissingval':
        return mano.bar(dataset)
    elif x == 'correlation':
        return mano.heatmap(dataset, figsize=(12,6))
    elif x == 'dendo':
        return mano.dendrogram(dataset)

In [35]:
def remspace(dataset):
    colname=str(input("Which column do you want to remove white space in?\n"))
    where = str(input("where do you want to remove the whitespace from?"))
    if colname == "":
        if where == "":
            dataset = dataset.str.replace(' ', '')
        if where == "beginning":
            dataset = dataset.str.lstrip()
        if where == "end":
            dataset = dataset.str.rstrip()
        if where == "both":
            dataset = dataset.str.strip()
    else:
        if where == "":
            dataset[colname] = dataset[colname].str.replace(' ', '')
        if where == "beginning":
            dataset[colname] = dataset[colname].str.lstrip()
        if where == "end":
            dataset[colname] = dataset[colname].str.rstrip()
        if where == "both":
            dataset[colname] = dataset[colname].str.strip()
    return dataset

def repspace(dataset):
    colname=str(input("Which column do you want to replace white space in?\n"))
    newvalue = str(input("what do you want to replace the white space with?"))
    where = str(input("where do you want to put the new value?"))
    if colname !="":
        if where == "beginning":
            dataset[colname].str.replace('^ +', newvalue)
        elif where == "end":
            dataset[colname].str.replace(' +$', newvalue)
        elif where == "both":
            dataset[colname].str.replace('^ +| +$', newvalue)
        elif where == "":
            dataset[colname].str.replace(' ', newvalue)
    else:
        if where == "beginning":
            dataset.str.replace('^ +', newvalue)
        elif where == "end":
            dataset.str.replace(' +$', newvalue)
        elif where == "both":
            dataset.str.replace('^ +| +$', newvalue)
        elif where == "":
            dataset.str.replace(' ', newvalue)
    return dataset

def extdmy(dataset):
    dataset = pd.DataFrame(dataset)
    colname = str(input("What column do you want to plot against time?"))
    dataset['year'] = dataset.index.year
    dataset['month'] = dataset.index.month
    dataset['day']=dataset.index.day
    dataset['weekday']=dataset.index.day_name()
    dataset['hour']=dataset.index.hour
    dataset['minute']=dataset.index.minute
    dataset['second']=dataset.index.second
    f, axes = plt.subplots(1,3)
    sb.boxplot(data=dataset, x = 'weekday', y = colname, ax = axes[0]) 
    sb.boxplot(data=dataset, x = 'month', y = colname, ax = axes[1])
    sb.boxplot(data=dataset, x = 'year', y = colname, ax = axes[2])
    return dataset


In [43]:
def description(name, rows):
    print("The columns and rows of the dataset are", name.shape)
    print("The number of values in the dataset is", name.size)
    print(name.head(rows))
    print(name.tail(rows))
    print(name.dtypes)
    print(name.columns)
    return name.describe()

In [34]:
def remcol(dataset):
    colname = str(input("which column do you want to remove?\n"))
    if colname != "":
        del(dataset[colname])
        return remcol(dataset)
    else:
        return dataset

In [45]:
def rename(dataset):
    colname = str(input("Which column do you want to rename?\n"))
    if colname!='':
        newcolname = str(input("New name of the column?\n"))
        dataset.rename(columns = {colname:newcolname}, inplace = True)
        return rename(dataset)

In [21]:
def remrow(dataset):
    colname = str(input("which column values do you want to remove?\n"))
    if colname == 'all':
        remrow.dataset = dataset.dropna(axis=0)
    elif colname =="":
        row = (input("which row/s do you want to remove?\n"))
        remrow.dataset = dataset[dataset[colname]!=row]
        return remrow(dataset)
    else:
        return remrow.dataset

In [3]:
def loadfile():
    filetype = str(input("What is the file type? (csv or excel)\n"))
    enc = str(input("What encoding do you want? (latin1 or unicode_escape)\n"))
    if filetype == 'csv':
        filepath = str(input("What is the filepath:"))
        data = pd.read_csv(filepath, index_col = None, encoding = enc)
        return data
    elif filetype == 'excel':
        filepath = str(input("What is the filepath:"))
        data = pd.read_excel(filepath, index_col = None)
        return data
    else:
        return None
    

In [48]:
def null(dataset):    
    return dataset.isnull().sum()

In [10]:
def fillmissingvals(dataset):
    method = str(input("which method do you want to use? (KNN, SimpleImp, Interpolation, mean, median, mode)"))
    colname = str(input("which column do you want to fill?"))
    if method == 'mode':
        dataset[colname].value_counts()
        mode = input("what is the most occurring value?")
        dataset[colname].fillna(mode, inplace=True)
    elif method == 'mean':
        dataset[colname].fillna(data[colname].mean)
    elif method == 'median':
        dataset[colname].fillna(data[colname].median)
    elif method == 'knn':
        colname1 = str(input('Which columns do you want to use?'))
        colname2=str(input('Which columns do you want to use?')) 
        neighbours = int(input("how many neighbours?\n"))
        imputer = KNNImputer(n_neighbors=neighbours)
        dataset[[colname1, colname2]] = imputer.fit_transform(dataset[[colname1, colname2]])
        return dataset
    elif method == 'simpleimp':
        newcolname = str(input("what is the new column name?\n"))
        imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
        dataset[[newcolname]] = imp_mean.fit(dataset[[colname]])
    elif method == 'Interpol':
        submethod = str(input("backwards or forwards?"))
        newcolname = str(input("what is the new column name?\n"))
        if submethod == 'forwards':
            dataset[newcolname] = dataset[colname].interpolate(method ='linear', limit_direction ='forward')
        elif submethod == 'backwards':
            dataset[newcolname] = dataset[colname].interpolate(method ='linear', limit_direction ='backward')

In [20]:
def changedtype(dataset):
    colname = str(input("which column do you want to change the data type for? Press enter if you want to change the data type for the entire dataset. "))
    if colname != '':
        changetype = str(input("what data type do you want to set the column to? (string, numbers, datetime)"))
        if changetype == 'string':
            dataset[colname]= dataset[colname].astype(str)
        elif changetype == 'float':
            dataset[colname]= dataset[colname].astype(float) #converting to float to accomodate all types of numbers
        elif changetype == 'boolean':
            dataset[colname]= dataset[colname].astype(bool)
        elif changetype == 'datetime':
            dataset[colname]=pd.to_datetime(dataset[colname])
        elif changetype == 'numbers':
            dataset[colname]=dataset[colname].astype(int)
    else:
        changetype = str(input("what data type do you want to set the column to? (string, numbers, datetime)"))
        if changetype == 'string':
            dataset= dataset.astype(str)
        elif changetype == 'float':
            dataset= dataset.astype(float) #converting to float to accomodate all types of numbers
        elif changetype == 'boolean':
            dataset= dataset.astype(bool)
        elif changetype == 'datetime':
            dataset=pd.to_datetime(dataset)
        elif changetype == 'numbers':
            dataset=dataset.astype(int)
    return dataset

In [28]:
def encoding(dataset):
    print(dataset.dtypes)
    method = str(input("what do you want to do?"))
    if method == "onehot":
        colname = str(input("which column do you want to encode?"))
        if colname != '':
            results = ohot.fit_transform(dataset[colname])
            newdat = pd.DataFrame(results.toarray(), columns = ohot.categories_)
            dataset = dataset.join(newdat)
        else: 
            results = ohot.fit_transform(dataset)
            newdat = pd.DataFrame(results.toarray(), columns = ohot.categories_)
            return newdat
    elif method == 'onehot2':
        df = pd.get_dummies(dataset)
        return df
    elif method == "ordinal":
        colname = str(input("which column do you want to encode?"))
        if colname !='':
            newcolname = str(input("set a new column name for encoded values"))
            dataset[newcolname] = ordenc.fit_transform(dataset[colname])
        else:
            return ordenc.fit_transform(dataset)
    elif method == 'labelencoder':
        le = preprocessing.LabelEncoder()
        le.fit(dataset[i])
        dataset = le.transform(dataset)
    return dataset
    

In [52]:
def KNNalgo(X_train, X_test, y_train, y_test):  
    neighbours = int(input("How many neighbours do you want?"))
    classifier = KNC(n_neighbors=neighbours)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print(confm(y_test, y_pred))
    print(crep(y_test, y_pred))
    print("These are the predicted values: ", y_pred)
    confmatrix = confm(y_test, y_pred)
    print("The Confusion Matrix is\n", confmatrix)
    Accuracy_Score = acs(y_test, y_pred)*100
    print("The Accuracy Score is\n", Accuracy_Score)
    precision = pscore(y_test, y_pred, pos_label = '1', labels = [0,1])*100
    print("The Precision is", precision)
    CReport = crep(y_test, y_pred)
    print("The Classification Report is\n", CReport)
    LogLoss = logloss(y_test, y_pred)
    print("The LogLoss is", LogLoss)
    fpr, tpr, _ = roccurve(y_test, y_pred)
    AreaUnderCurve = auc(fpr, tpr)
    print("The Area Under the Curve is", AreaUnderCurve)
    RecallScore = rscore(y_test, y_pred,pos_label='1',labels=[0,1])*100
    print("The Recall Score is", RecallScore)
    print("The Specificity is", fpr, "and the Sensitivity is", tpr)
    FScore = f1score(y_test, y_pred)
    print("The F-Score is", FScore)

In [14]:
def NVB(X_train, X_test, y_train, y_test):
    y_pred = GNB().fit(X_train, y_train).predict(X_test)
    print("These are the predicted values: ", y_pred)
    print("The number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != y_pred).sum()))
    confmatrix = confm(y_test, y_pred)
    print("The Confusion Matrix is\n", confmatrix)
    Accuracy_Score = acs(y_test, y_pred)*100
    print("The Accuracy Score is\n", Accuracy_Score)
    precision = pscore(y_test, y_pred, pos_label = '1', labels = [0,1])*100
    print("The Precision is", precision)
    CReport = crep(y_test, y_pred)
    print("The Classification Report is\n", CReport)
    LogLoss = logloss(y_test, y_pred)
    print("The LogLoss is", LogLoss)
    fpr, tpr, _ = roccurve(y_test, y_pred)
    AreaUnderCurve = auc(fpr, tpr)
    print("The Area Under the Curve is", AreaUnderCurve)
    RecallScore = rscore(y_test, y_pred,pos_label='1',labels=[0,1])*100
    print("The Recall Score is", RecallScore)
    print("The Specificity is", fpr, "and the Sensitivity is", tpr)
    FScore = f1score(y_test, y_pred)
    print("The F-Score is", FScore)

In [15]:
def Perceptron(X_train, X_test, y_train, y_test):
    clf = mlpc(tol=1e-3, random_state=0)
    y_pred = clf.fit(X_train, y_train).predict(X_test)
    print("These are the predicted values: ", y_pred)
    print('The Perceptron score is', clf.score(X_train, y_train))
    confmatrix = confm(y_test, y_pred)
    print("The Confusion Matrix is\n", confmatrix)
    Accuracy_Score = acs(y_test, y_pred)*100
    print("The Accuracy Score is\n", Accuracy_Score)
    precision = pscore(y_test, y_pred, pos_label = '1', labels = [0,1])*100
    print("The Precision is", precision)
    CReport = crep(y_test, y_pred)
    print("The Classification Report is\n", CReport)
    LogLoss = logloss(y_test, y_pred)
    print("The LogLoss is", LogLoss)
    fpr, tpr, _ = roccurve(y_test, y_pred)
    AreaUnderCurve = auc(fpr, tpr)
    print("The Area Under the Curve is", AreaUnderCurve)
    RecallScore = rscore(y_test, y_pred,pos_label='1',labels=[0,1])*100
    print("The Recall Score is", RecallScore)
    print("The Specificity is", fpr, "and the Sensitivity is", tpr)
    FScore = f1score(y_test, y_pred)
    print("The F-Score is", FScore)

In [2]:
def supvec(X_train, X_test, y_train, y_test):
    knl = str(input('Which kernel do you want to apply? (linear, rbf)'))
    c= int(input('Value of c?' ))
    clf = svm.SVC(kernel = knl, C = c, random_state = 42)
    y_pred = clf.fit(X_train, y_train).predict(X_test)
    print("These are the predicted values: ", y_pred)
    confmatrix = confm(y_test, y_pred)
    print("The Confusion Matrix is\n", confmatrix)
    Accuracy_Score = acs(y_test, y_pred)*100
    print("The Accuracy Score is\n", Accuracy_Score)
    precision = pscore(y_test, y_pred, average = 'macro', labels = [0,1])*100
    print("The Precision is", precision)
    CReport = crep(y_test, y_pred)
    print("The Classification Report is\n", CReport)
    LogLoss = logloss(y_test, y_pred)
    print("The LogLoss is", LogLoss)
    fpr, tpr, _ = roccurve(y_test, y_pred)
    AreaUnderCurve = auc(fpr, tpr)
    print("The Area Under the Curve is", AreaUnderCurve)
    RecallScore = rscore(y_test, y_pred, average = 'macro',labels=[0,1])*100
    print("The Recall Score is", RecallScore)
    print("The Specificity is", fpr, "and the Sensitivity is", tpr)
    FScore = f1score(y_test, y_pred)
    print("The F-Score is", FScore)
    
def RandFor(X_train, X_test, y_train, y_test):
    y_pred = RFC().fit(X_train, y_train).predict(X_test)
    print("These are the predicted values: ", y_pred)
    confmatrix = confm(y_test, y_pred)
    print("The Confusion Matrix is\n", confmatrix)
    Accuracy_Score = acs(y_test, y_pred)*100
    print("The Accuracy Score is\n", Accuracy_Score)
    precision = pscore(y_test, y_pred, average = 'macro', labels = [0,1])*100
    print("The Precision is", precision)
    CReport = crep(y_test, y_pred)
    print("The Classification Report is\n", CReport)
    LogLoss = logloss(y_test, y_pred)
    print("The LogLoss is", LogLoss)
    fpr, tpr, _ = roccurve(y_test, y_pred)
    AreaUnderCurve = auc(fpr, tpr)
    print("The Area Under the Curve is", AreaUnderCurve)
    RecallScore = rscore(y_test, y_pred,average = 'macro',labels=[0,1])*100
    print("The Recall Score is", RecallScore)
    print("The Specificity is", fpr, "and the Sensitivity is", tpr)
    FScore = f1score(y_test, y_pred)
    print("The F-Score is", FScore)
    
def ada(X_train, X_test, y_train, y_test):
    y_pred = ABC().fit(X_train, y_train).predict(X_test)
    print("These are the predicted values: ", y_pred)
    confmatrix = confm(y_test, y_pred)
    print("The Confusion Matrix is\n", confmatrix)
    Accuracy_Score = acs(y_test, y_pred)*100
    print("The Accuracy Score is\n", Accuracy_Score)
    precision = pscore(y_test, y_pred, average = 'macro', labels = [0,1])*100
    print("The Precision is", precision)
    CReport = crep(y_test, y_pred)
    print("The Classification Report is\n", CReport)
    LogLoss = logloss(y_test, y_pred)
    print("The LogLoss is", LogLoss)
    fpr, tpr, _ = roccurve(y_test, y_pred)
    AreaUnderCurve = auc(fpr, tpr)
    print("The Area Under the Curve is", AreaUnderCurve)
    RecallScore = rscore(y_test, y_pred,average = 'macro',labels=[0,1])*100
    print("The Recall Score is", RecallScore)
    print("The Specificity is", fpr, "and the Sensitivity is", tpr)
    FScore = f1score(y_test, y_pred)
    print("The F-Score is", FScore)

def boostaway(X_train, X_test, y_train, y_test):
    xg_reg = XGBC(disable_default_eval_metric=1)
    xg_reg.fit(X_train,y_train)
    y_pred = xg_reg.predict(X_test)
    print("These are the predicted values: ", y_pred)
    confmatrix = confm(y_test, y_pred)
    print("The Confusion Matrix is\n", confmatrix)
    Accuracy_Score = acs(y_test, y_pred)*100
    print("The Accuracy Score is\n", Accuracy_Score)
    precision = pscore(y_test, y_pred,  average = 'macro')*100
    print("The Precision is", precision)
    CReport = crep(y_test, y_pred)
    print("The Classification Report is\n", CReport)
    LogLoss = logloss(y_test, y_pred)
    print("The LogLoss is", LogLoss)
    fpr, tpr, _ = roccurve(y_test, y_pred)
    AreaUnderCurve = auc(fpr, tpr)
    print("The Area Under the Curve is", AreaUnderCurve)
    RecallScore = rscore(y_test, y_pred, average = 'macro')*100
    print("The Recall Score is", RecallScore)
    FScore = f1score(y_test, y_pred)
    print("The F-Score is", FScore)
    return y_pred

def light(X_train, X_test, y_train, y_test):
    X_train = StdSclr().fit_transform(X_train, y_train)
    X_test = StdSclr().fit_transform(X_test)
    clf = clf = lgbm.LGBMClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print("These are the predicted values: ", y_pred)
    confmatrix = confm(y_test, y_pred)
    print("The Confusion Matrix is\n", confmatrix)
    Accuracy_Score = acs(y_test, y_pred)*100
    print("The Accuracy Score is\n", Accuracy_Score)
    precision = pscore(y_test, y_pred, pos_label = '1', labels = [0,1])*100
    print("The Precision is", precision)
    CReport = crep(y_test, y_pred)
    print("The Classification Report is\n", CReport)
    LogLoss = logloss(y_test, y_pred)
    print("The LogLoss is", LogLoss)
    fpr, tpr, _ = roccurve(y_test, y_pred)
    AreaUnderCurve = auc(fpr, tpr)
    print("The Area Under the Curve is", AreaUnderCurve)
    RecallScore = rscore(y_test, y_pred,pos_label='1',labels=[0,1])*100
    print("The Recall Score is", RecallScore)
    print("The Specificity is", fpr, "and the Sensitivity is", tpr)
    FScore = f1score(y_test, y_pred)
    print("The F-Score is", FScore)

def LogisticRegression(X_train, X_test, y_train, y_test):
    y_pred = LogR().fit(X_train, y_train).predict(X_test)
    print("These are the predicted values: ", y_pred)
    confmatrix = confm(y_test, y_pred)
    print("The Confusion Matrix is\n", confmatrix)
    Accuracy_Score = acs(y_test, y_pred)*100
    print("The Accuracy Score is\n", Accuracy_Score)
    precision = pscore(y_test, y_pred, average = 'macro', labels = [0,1])*100
    print("The Precision is", precision)
    CReport = crep(y_test, y_pred)
    print("The Classification Report is\n", CReport)
    LogLoss = logloss(y_test, y_pred)
    print("The LogLoss is", LogLoss)
    fpr, tpr, _ = roccurve(y_test, y_pred)
    AreaUnderCurve = auc(fpr, tpr)
    print("The Area Under the Curve is", AreaUnderCurve)
    RecallScore = rscore(y_test, y_pred,average = 'macro',labels=[0,1])*100
    print("The Recall Score is", RecallScore)
    print("The Specificity is", fpr, "and the Sensitivity is", tpr)
    FScore = f1score(y_test, y_pred)
    print("The F-Score is", FScore)
    
    
def DecTree(X_train, X_test, y_train, y_test):
    clf= dtc()
    y_pred = clf.fit(X_train, y_train).predict(X_test)
    print("These are the predicted values: ", y_pred)
    confmatrix = confm(y_test, y_pred)
    print("The Confusion Matrix is\n", confmatrix)
    Accuracy_Score = acs(y_test, y_pred)*100
    print("The Accuracy Score is\n", Accuracy_Score)
    precision = pscore(y_test, y_pred, pos_label = '1', labels = [0,1])*100
    print("The Precision is", precision)
    CReport = crep(y_test, y_pred)
    print("The Classification Report is", CReport)
    LogLoss = logloss(y_test, y_pred)
    print("The LogLoss is", LogLoss)
    fpr, tpr, _ = roccurve(y_test, y_pred)
    AreaUnderCurve = auc(fpr, tpr)
    print("The Area Under the Curve is", AreaUnderCurve)
    RecallScore = rscore(y_test, y_pred,pos_label='1',labels=[0,1])*100
    print("The Recall Score is", RecallScore)
    print("The Specificity is", fpr, "and the Sensitivity is", tpr)
    FScore = f1score(y_test, y_pred)
    print("The F-Score is", FScore)

In [56]:
def splitting(dataset):
    y=str(input("What is the label column"))
    label=dataset[y]
    X=dataset.drop(columns=[y])
    X_train, X_test, y_train, y_test = tts(X, label, test_size=0.33, random_state=42)
    return X_train, X_test, y_train, y_test

In [17]:
def regularize(X_train, y_train, X_test, y_test, X, y):
    method = str(input("Which regularization do you want to implement (lasso, ridge, lin regression, log regression)?"))
    scaler = StdSclr(copy=True, with_mean=True, with_std=True)
    X_train = scaler.fit(X_train.fillna(0))
    if method == 'lasso':
        cross_val_scores_lasso = []
        alpha = []
        for i in range(1, 9):
            model = lso(alpha = i * 0.25, tol=0.0925)
            model.fit(X_train, y_train)
            scores = cvl(model, X,y, cv = 10)
            avg_cross_val_score = mean(scores)*100
            cross_val_scores_lasso.append(avg_cross_val_score)
            alpha.append(i * 0.25)
        for i in range(0, len(alpha)):
            print(str(alpha[i])+' : '+str(cross_val_scores_lasso[i]))
        chosen = lso(alpha = alpha[cross_val_scores_lasso.index(max(cross_val_scores_lasso))], tol = 0.0925)
        chosen.fit(X_train, y_train)
        print('The regularization score of the Lasso model is', chosen.score(X_test, y_test))
        y_pred = chosen.fit(X_train, y_train).predict(X_test)
        print("These are the predicted values: ", y_pred)
    if method == 'ridge':
        cross_val_scores_ridge = []
        alpha = []
        for i in range(1, 9):
            model = rdg(alpha = i * 0.25)
            model.fit(X_train, y_train)
            scores = cvl(model, X, y, cv = 10)
            avg_cross_val_score = mean(scores)*100
            cross_val_scores_ridge.append(avg_cross_val_score)
            alpha.append(i * 0.25)
        for i in range(0, len(alpha)):
            print(str(alpha[i])+' : '+str(cross_val_scores_ridge[i]))
        chosen = rdg(alpha = alpha[cross_val_scores_ridge.index(max(cross_val_scores_ridge))])
        chosen.fit(X_train, y_train)
        print('The regularization score of the Ridge model is', chosen.score(X_test, y_test))
        y_pred = chosen.fit(X_train, y_train).predict(X_test)
        print("These are the predicted values: ", y_pred)
    if method == 'log regression':
        model = LogR()
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        print('The regularization score with Logarithmic Regression is', score, '.')
        y_pred = model.fit(X_train, y_train).predict(X_test)
    if method == 'lin regression':
        model = LR()
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        print('The regularization score with Linear Regression is', score, '.')
        y_pred = model.fit(X_train, y_train).predict(X_test)
        print("These are the predicted values: ", y_pred)
    return y_pred

In [62]:
def crossvalscore(X_train, X_test, y_train, y_test):
    testsize = float(input('What do you want the test size to be? Please input in decimal form.'))
    knl = str(input('Which kernel do you want to apply? (linear, rbf)'))
    clf = XGBC(disable_default_eval_metric=1)
    score = cvl(clf, X_train, y_train, cv=5)
    print('The normal cross validation score is', score)
    print("For the normal cross validation score, there is a %0.2f accuracy with a standard deviation of %0.2f" % (score.mean(), score.std()))
    sscore = cvl(clf, X_train, y_train, cv=SSS(n_splits = 5, test_size=testsize, random_state=0))
    print('The ShuffleSplit score is', sscore)
    print("For the ShuffleSplit cross validation score, there is a %0.2f accuracy with a standard deviation of %0.2f" % (sscore.mean(), sscore.std()))

In [63]:
def crossval(dataset):
    method = str(input("What Fold method do you want to use?(kfold, rkfold, skfold, lvo, sss, rskfold)"))
    if method == 'kfold':
        k = int(input("What number-fold cross validation do you want to do?"))
        kf= KFold(n_splits=k)
        for train_index, test_index in kf.split(dataset):
            trainX, testX = X.iloc[train_index], X.iloc[test_index] 
            trainY, testY = y.iloc[train_index], y.iloc[test_index]
            return trainX, testX, trainY, testY
    if method == 'rkfold':
        k = int(input("What number-fold cross validation do you want to do?"))
        r = int(input("How many times do you want to repeat the cross validation?"))
        kf = RKFold(n_splits=k, n_repeats=r, random_state=random_state)
        for train_index, test_index in kf.split(dataset):
            trainX, testX = X.iloc[train_index], X.iloc[test_index] 
            trainY, testY = y.iloc[train_index], y.iloc[test_index]
            return trainX, testX, trainY, testY
    if method == 'skfold':
        k = int(input("What number-fold cross validation do you want to do?"))
        kf = SKFold(n_splits=k, random_state=None, shuffle=False)
        label = str(input("What is the label of the dataset?"))
        y=dataset[label]
        X=dataset.drop([label], axis=1)
        kf.get_n_splits(X,y)
        print(kf)
        for train_index, test_index in kf.split(X,y):
            trainX, testX = X.iloc[train_index], X.iloc[test_index] 
            trainY, testY = y.iloc[train_index], y.iloc[test_index]
            return trainX, testX, trainY, testY
    if method == 'rskfold':
        k = int(input("What number-fold cross validation do you want to do?"))
        r = int(input("How many times do you want to repeat the cross validation?"))
        kf = rskfold(n_splits=k, n_repeats=r, random_state=36851234)
        label = str(input("What is the label of the dataset?"))
        y=dataset[label]
        X=dataset.drop([label], axis=1)
        kf.get_n_splits(X,y)
        print(kf)
        for train_index, test_index in kf.split(X,y):
            trainX, testX = X.iloc[train_index], X.iloc[test_index] 
            trainY, testY = y.iloc[train_index], y.iloc[test_index]
            return trainX, testX, trainY, testY
    if method == 'lvo':
        for train_index, test_index in lvo.split(dataset):
            trainX, testX = X.iloc[train_index], X.iloc[test_index] 
            trainY, testY = y.iloc[train_index], y.iloc[test_index]
            return trainX, testX, trainY, testY

In [64]:
def feature_selection(X,y):
    estimator = SVR(kernel="linear")
    selector = RFE(estimator, n_features_to_select=5, step=1)
    selector = selector.fit(X, y)
    print(selector.support_)
    print(selector.ranking_)

In [65]:
def ANOVA(dataset):
    colname1 = str(input("first column you want to relate"))
    colname2 = str(input("second column you want to relate"))
    covariance = np.cov(dataset[colname1], dataset[colname2])
    print("The Covariance is ",covariance)
    
    corr1, _ = spm(dataset[colname1], dataset[colname2])
    print('Spearman correlation: %.2f' % corr1)
    corr2, _ = pe(dataset[colname1], dataset[colname2])
    print('Pearsons correlation: %.2f' % corr2)
    print('The t-test value is', (dataset[colname1].mean()-dataset[colname2].mean())/(math.sqrt((dataset[colname1].std()**2)/dataset[colname1].count() + (dataset[colname2].std()**2)/dataset[colname2].count())))
    obs = np.array(dataset)
    obs[obs<0]=0
    chi2_contingency(obs)
    g, p, dof, expctd = chi2_contingency(obs, lambda_="log-likelihood")
    print('The critical value of Chi-Square is', g) 
    print('The p-value is', p)
    print('The degrees of freedom are', dof)
    print('The expected values using the Chi-Square method are', expctd)
    alpha = 0.05
    if p <= alpha:
        print('Dependent (reject H0)')
    else:
        print('Independent (H0 holds true)')
    return dataset.plot(x= colname1,y=colname2,kind='scatter',alpha=0.5,cmap='rainbow')

def clheatmap(dataset):
    corrmatrix = dataset.corr()
    colour = str(input("Heatmap colour?\n"))
    clmap = sb.clustermap(corrmatrix, cmap = colour, linewidths = 0.2); 
    f, axis = plt.subplots(figsize =(15, 10)) 
    plt.setp(clmap.ax_heatmap.yaxis.get_majorticklabels(), rotation = 0) 
    return clmap


def correlation_analysis(dataset):
    numcols = dataset.select_dtypes(include=np.number)
    corr = numcols.corr()
    ax = sb.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sb.diverging_palette(20, 220, n=200),
    square=True
    )
    
    ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right')


In [66]:
def numanalysis(dataset):
    colname = str(input("which column do you want to analyze? Press enter if you want to analyze the entire dataset. "))
    analysis_type = str(input("What analysis do you want to do? (bar charts, value counts?(barchart, value counts))"))
    if analysis_type == 'value counts':
        if colname != '':
            return dataset[colname].value_counts()
        else:
            return dataset.value_counts()
    elif analysis_type == 'barchart':
        if colname != '':
            return dataset[colname].plot.bar()
        else:
            return dataset[colname].plot.bar()

In [68]:
def Usampler(X,y):
    print('Original dataset shape %s' % Counter(y))
    rus = RandomUnderSampler(random_state=42)
    X_res, y_res = rus.fit_resample(X, y)
    print('Resampled dataset shape %s' % Counter(y_res))
    return X_res, y_res

In [69]:
def Osampler(X,y):
    print('Original dataset shape %s' % Counter(y))
    ros = RandomOverSampler(random_state=42)
    X_res, y_res = ros.fit_resample(X, y)
    print('Resampled dataset shape %s' % Counter(y_res))
    return X_res, y_res

# TSF Template

In [57]:
def loadseries():
    name = str(input('what is the name of file?'))
    return pd.read_csv(name, header=0, index_col=0, parse_dates=True, squeeze=True)

In [2]:
def lagset(dataset):
    dataset = pd.DataFrame(dataset)
    steps = int(input('How many steps do you want to lag the dataset?'))
    newdata = dataset.shift(steps)
    newdata = pd.concat([newdata, dataset], axis=1)
    newdata.fillna(0, inplace=True)
    return newdata

In [4]:
def rollingmean2(dataset):
    data = pd.DataFrame(dataset.values)
    window2 = data.shift(2).rolling(window=2)
    means=window2.mean()
    rwindow2 = pd.concat([means, data],axis=1)
    rwindow2.columns = ['mean(t-2,t-1)', 't']
    rwindow2.fillna(0, inplace=True)
    return rwindow2
    

In [5]:
def rollingmean3(dataset):
    data = pd.DataFrame(dataset.values)
    window3 = data.shift(3).rolling(window=3)
    avg = window3.mean()
    rwindow3 = pd.concat([avg, data], axis=1)
    rwindow3.columns = ['mean(t-3, t-2, t-1)', 't']
    rwindow3.fillna(0, inplace=True)
    return rwindow3

In [ ]:
def expandingwindow(dataset):
    expwindow = dataset.expanding()
    dataframe = pd.concat([expwindow.min(), expwindow.mean(), expwindow.max(), shampsales.shift(-1)], axis=1)
    dataframe.columns = ['min', 'mean', 'max', 't']
    print(dataframe.head(5))
    return dataframe

In [41]:
def resample(dataset):
    frequency = str(input("What is your resampling frequency? (quarterly, months, weeks, days, hours, minutes, seconds?)"))
    if frequency == 'quarterly':
        resmpdata = dataset.resample('Q').mean()
    elif frequency == 'months':
        resmpdata = dataset.resample('M').mean()
    elif frequency == 'weeks':
        resmpdata = dataset.resample('W').mean()
    elif frequency == 'days':
        resmpdata = dataset.resample('D').mean()
    elif frequency == 'hours':
        resmpdata = dataset.resample('H').mean()
    elif frequency == 'minutes':
        resmpdata = dataset.resample('T').mean()
    elif frequency == 'seconds':
        resmpdata = dataset.resample('S').mean()
    else:
        print("error")
        return None 
    return resmpdata

In [55]:
def interpolseriesdata(dataset):
    interpoldata = dataset.interpolate(method='linear')
    print(interpoldata.head())
    interpoldata.plot(marker = 'X', linestyle = '-', alpha = 0.8, figsize = (20,8), color='purple')
    plt.show()
    return interpoldata

In [112]:
def seasonaldecompose(dataset):
    colname = str(input("What column do you want to decompose?"))
    timeperiod = str(input("What time period do you want to decompose the data over? (months, weeks, days)"))
    if timeperiod == 'months':
        result = seasonal_decompose(dataset[colname], model='additive', freq=12)
    elif timeperiod == 'weeks':
        result = seasonal_decompose(dataset[colname], model='additive', freq=52)
    elif timeperiod == 'days':
        result = seasonal_decompose(dataset[colname], model='additive', freq=365)
    else:
        print("error")
    return result.plot()

In [2]:
def stationaritycheck(dataset):
    colname = str(input("What column do you want to check the stationarity for?"))
    X = dataset[colname].values
    split=int(len(X)/2)
    X1,X2=X[0:split],X[split:]
    mean1, mean2=X1.mean(),X2.mean()
    var1, var2= X1.var(),X2.var()
    print('mean1=%f,\nmean2=%f'%(mean1,mean2))
    print('var1=%f,\nvar2=%f'%(var1,var2))
    variance1 = abs(mean1-mean2)
    variance2 = abs(var1-var2)
    if variance1 > 0:
        print("There is a difference of", variance1, "between the means of the two halves of the dataset.")
    else:
        print("The data is stationary.")
    if variance2 > 0:
        print("There is a difference of", variance2, "between the variances of the two halves of the dataset.")
    else:
        print("The data is stationary.")
    ADF = str(input("Do you want to administer the ADFuller test for further clarity? (Yes (y) or No (n))"))
    if ADF == 'n':
        return
    else:
        result1 = adfuller(X)
        print('ADF Statistic for basic data: %f' % result1[0])
        if result1[0]>1:
            print('Very likely that null hypothesis is true, meaning we have a non-stationary dataset')
        elif result1[0]<0:
            print('Very likely that null hypothesis is false, meaning we have a stationary dataset')
        else:
            print('Hard to say if null hypothesis is true or false, it is complicated')
        print('p-value : %f'%result1[1])
        print(' Critical Values:\n        Key   Value')
        for key , value in result1[4].items():
            print('\t%s: %.6f' % (key, value))
        print('\n')
        
        result2 = adfuller(np.log(X))
        print('ADF Statistic for log transformed data: %f' % result2[0])
        if result2[0]>1:
            print('Very likely that null hypothesis is true, meaning we have a non-stationary dataset')
        elif result2[0]<0:
            print('Very likely that null hypothesis is false, meaning we have a stationary dataset')
        else:
            print('Hard to say if null hypothesis is true or false, it is complicated')
        print('p-value : %f'%result2[1])
        print(' Critical Values:\n        Key   Value')
        for key , value in result2[4].items():
            print('\t%s: %.6f' % (key, value))
        print('\n')
        
        result = adfuller(np.sqrt(X))
        print('ADF Statistic for square root transformed data: %f' % result[0])
        if result[0]>1:
            print('Very likely that null hypothesis is true, meaning we have a non-stationary dataset')
        elif result[0]<0:
            print('Very likely that null hypothesis is false, meaning we have a stationary dataset')
        else:
            print('Hard to say if null hypothesis is true or false, it is complicated')
        print('p-value : %f'%result[1])
        print(' Critical Values:\n        Key   Value')
        for key , value in result[4].items():
            print('\t%s: %.6f' % (key, value))

In [ ]:
def boxcoxtransformation(dataset):
    colname = str(input("What column do you want to transform?"))
    lambdavalue = int(input("Set a lambda value which transforms the data closest to a normal distribution. (What transformation gives the best ADFuller test result?)"))
    dataset[colname]=boxcox(dataset[colname],lmbda=lambdavalue) 
    plt.figure(figsize=(12,8))
    plt.subplot(211)
    plt.plot(dataset[colname])
    plt.subplot(212)
    plt.hist(dataset[colname],edgecolor='black')

In [7]:
def differencing(dataset, interval):
    differenced = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        differenced.append(value)
    return pd.Series(differenced)

def inversedifferencing(history, yhat, interval):
    newdata = yhat + history[-interval]
    return newdata

def inversetransformation(differenceddataset, originaldata, yhat, interval):
    inverteddataset = list()
    for i in range(len(differenceddataset)):
        value = inversedifferencing(originaldata, yhat, interval)
        inverteddataset.append(value)
    inverted = pd.Series(inverteddataset)
    print(inverted.head(10))
    return inverted

In [9]:
def scaling(series):
    X = series.values.reshape(len(X),1)
    scaler = MMS(feature_range = (-1,1)).fit(X)
    transformation = str(input("Type 's' for normal scaling and 'i' for inverse scaling."))
    if transformation == 's':
        scaled = scaler.transform(X)
        scaled = pd.series(scaled[:,0])
        print(scaled.head())
        return scaled
    else:
        invertedscaled = scaler.inverse_transform(scaled)
        inverted_series = pd.Series(invertedscaled[:, 0])
        print(inverted_series.head(10))
        return inverted_series

In [ ]:
def checkcorrelation(dataset):
    colname = str(input("Which column do you want to plot?"))
    plt.figure(figsize = (11,5))
    lag_plot(dataset[colname])
    acplot(dataset[colname])
    plot_acf(dataset[colname], lags=30)
    plt.show()

In [20]:
def autoregression(dataset):
    colname = str(input("Which column do you want to analyze?"))
    data = dataset[colname].values
    train, test = data[0:len(data)*0.66], data[len(data)*0.66:]
    train_X, train_y = train[:,0], train[:,1]
    test_X, test_y = test[:,0], test[:,1]
    pred = [x for x in data]
    train_resid = [train_y[i]-pred[i] for i in range(len(pred))]
    model = AR(train_resid).fit()
    window = model.k_ar
    prediction = model_fit.forecast(steps=5)
    coef = model_fit.params
    print('\nLag=%d \nCoef=%s' % (window, coef))
    history = train_resid[len(train_resid)-window:]
    history = [history[i] for i in range(len(history))]
    predictions = list()
    expected_error = list()
    for t in range(len(test_y)):
        yhat = test_X[t]
        error = test_y[t] - yhat
        expected_error.append(error)
        length = len(history)
        lag = [history[i] for i in range(length-window,length)]
        pred_error = coef[0]
        for d in range(window):
            pred_error += coef[d+1] * lag[window-d-1]
        predictions.append(pred_error)
        history.append(error)
        print('predicted error=%f, expected error=%f' % (pred_error, error))
    
    performance = MSE(test_y, predictions, squared = False)
    print("\n Test RMSE: %.4f" % performance)
    plt.plot(expected_error, 'red')
    plt.plot(test_y, color = 'blue')
    plt.plot(predictions, color = 'green')
    plt.show()

In [21]:
def ARIMAmodel(dataset, colname):
    model = ARIMA(dataset[colname], order=(3,1,3))
    model_fit = model.fit(disp=0)
    print(model_fit.summary())
    residuals = pd.DataFrame(model_fit.resid)
    print("\n Normal Residuals Plot")
    residuals.plot()
    plt.show()
    print("\n Kernel Density Residuals Plot")
    residuals.plot(kind='kde')
    plt.show()
    print("\n", residuals.describe())
    history = dataset[colname].values
    model2 = ARIMA(history, order=(2,1,1))
    model2_fit = model2.fit(disp=0)
    prediction = pd.DataFram(model_fit.forecast(steps=5))
    print("\n", prediction)
    print("\n Normal Predictions Plot")
    prediction.plot()
    plt.show()
    print("\n Kernel Density Predictions Plot")
    prediction.plot(kind='kde')
    plt.show()

## Building a Dataset from Raw File

In [26]:
#for reading a dictionary standard words UNIX file
def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [word for i in range(len(content)) for word in content[i].split()]
    content = np.array(content)
    return content

In [25]:
#building dictionary
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary